In [1]:
# folder paths
FILE_PATH = '/home/ubuntu/use_gpu/classification'
SAVE_PATH = '/home/ubuntu/use_gpu/classification/submit'

In [10]:
!pip install sentence-transformers

  Using cached sentence-transformers-2.2.2.tar.gz (85 kB)
  Preparing metadata (setup.py) ... done
  Using cached sentencepiece-0.1.99-cp310-cp310-macosx_11_0_arm64.whl (1.2 MB)
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125925 sha256=90ee2e8ca5d1dda95f4b450165911a468739544548bf07a49a9d6b9604b6e8df
  Stored in directory: /Users/jeonminjeong/Library/Caches/pip/wheels/0a/f5/dd/9d00836c4e9e279c2a59d5b0ab72dafa66cbc626a327c550dd
Successfully built sentence-transformers


In [2]:
# import

import re
import pandas as pd
import numpy as np
import random
from sentence_transformers import SentenceTransformer

import random
import torch
import os
import pandas as pd
from collections import Counter
from sklearn.preprocessing import MinMaxScaler

# clustering
from sklearn.cluster import KMeans
from sklearn.cluster import Birch
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import BisectingKMeans
from sklearn.mixture import GaussianMixture

In [3]:
# random seed

random_seed = 42

torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

In [4]:
# get data
df = pd.read_csv(os.path.join(FILE_PATH, 'news.csv'))

# 제목 + 내용
df['text'] = df['title'] + ' : ' + df['contents']

preprocessing

In [5]:
def preprocess_text(text):
    # URL 제거
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    
    # 해시태그 제거
    text = re.sub(r'#\w+', '', text)
    
    # 멘션 제거
    text = re.sub(r'@\w+', '', text)
    
    # 이모지 제거
    text = text.encode('ascii', 'ignore').decode('ascii')
    
    # 공백 및 특수문자 제거
    text = re.sub(r'\s+', ' ', text).strip()
    
    # 숫자 제거
    text = re.sub(r'\d+', '', text)
    
    return text.lower()


df['processed_text'] = df['text'].apply(preprocess_text)

select model and clustering algorithm

In [ ]:

'''
# 사용된 CSV 파일 목록

"st_submit_knn_allbert_scaler_title.csv", # sentence-transformers/all-distilroberta-v1
"st_submit_knn_allbert_scaler.csv", 
"st_submit_knn_mini12.csv", # sentence-transformers/all-MiniLM-L12-v2
'st_submit_bkm_mini12.csv', 
"st_submit_birch_mini12.csv", 
"st_submit_knn_mpnet.csv", # sentence-transformers/all-mpnet-base-v2
'st_submit_agglomerative.csv', # sentence-transformers/all-MiniLM-L6-v2
'st_submit_birch_pminilm.csv', # sentence-transformers/paraphrase-MiniLM-L6-v2
'st_submit_gmm_pminilm.csv', 
'st_submit_knn_pmini12.csv' # sentence-transformers/paraphrase-MiniLM-L12-v2


'ensemble_result_wo3_w24ensemble_wo2.csv',  # ['st_submit_knn_allbert_scaler.csv', 'st_submit_knn_mini12.csv', 'st_submit_birch_mini12.csv', 'st_submit_knn_allbert_scaler_title.csv', 'st_submit_bkm_mini12.csv', 'st_submit_agglomerative.csv']
'ensemble_result_wo3_w24ensemble_wo1.csv',  # ['st_submit_knn_allbert_scaler.csv', 'st_submit_knn_mpnet.csv', 'st_submit_birch_mini12.csv', 'st_submit_knn_allbert_scaler_title.csv', 'st_submit_bkm_mini12.csv', 'st_submit_agglomerative.csv']
'ensemble_result_wo3_w7_enall2.csv',        # ["st_submit_knn_allbert_scaler.csv", "st_submit_knn_mini12.csv", "st_submit_knn_mpnet.csv", "st_submit_birch_mini12.csv", "st_submit_knn_allbert_scaler_title.csv", 'st_submit_bkm_mini12.csv' , 'st_submit_agglomerative.csv', 'ensemble_result_wo3_w24ensemble_wo2.csv', 'ensemble_result_wo3_w24ensemble_wo1.csv', 'ensemble_result_wo3_w24ensemble_gmm.csv', 'ensemble_result_wo3_w24ensemble_wo5.csv']
'ensemble_result_wo3_w7_enall4.csv'         # ["st_submit_knn_allbert_scaler.csv", "st_submit_knn_mini12.csv", "st_submit_knn_mpnet.csv", "st_submit_birch_mini12.csv", "st_submit_knn_allbert_scaler_title.csv", 'st_submit_bkm_mini12.csv' , 'st_submit_agglomerative.csv', ]'ensemble_result_wo3_w24ensemble_wo2.csv', 'ensemble_result_wo3_w24ensemble_wo1.csv', 'ensemble_result_wo3_w24ensemble_gmm.csv'] 
'ensemble_result_wo3_w7_en0.csv'            #  ['st_submit_knn_allbert_scaler.csv', 'st_submit_knn_mini12.csv', 'st_submit_knn_mpnet.csv', 'st_submit_birch_mini12.csv', 'st_submit_knn_allbert_scaler_title.csv', 'st_submit_bkm_mini12.csv', 'st_submit_agglomerative.csv', 'ensemble_result_wo3_w24ensemble_wo2.csv', 'ensemble_result_wo3_w24ensemble_wo1.csv']


# 연달아 필요한 것
'st_submit_gmm_mini12.csv' # sentence-transformers/all-MiniLM-L12-v2
'ensemble_result_wo3_w24ensemble_gmm.csv' # ["st_submit_knn_allbert_scaler.csv", "st_submit_knn_mini12.csv", "st_submit_knn_mpnet.csv", "st_submit_birch_mini12.csv", "st_submit_knn_allbert_scaler_title.csv", 'st_submit_bkm_mini12.csv' , 'st_submit_agglomerative.csv', 'st_submit_gmm_mini12.csv']
'ensemble_result_wo3_w24ensemble_wo5.csv' # ['st_submit_knn_allbert_scaler.csv', 'st_submit_knn_mini12.csv', 'st_submit_knn_mpnet.csv', 'st_submit_birch_mini12.csv', 'st_submit_knn_allbert_scaler_title.csv', 'st_submit_agglomerative.csv']


'''

In [6]:
cluster_dict = {
    "kmeans_cluster": KMeans(n_clusters=6, random_state=random_seed),
    "birch_cluster": Birch(threshold=0.03, n_clusters=6),
    "agglo_cluster": AgglomerativeClustering(n_clusters=6),
    "gmm_cluster": GaussianMixture(n_components=6, random_state=random_seed),
    "bkm_cluster": BisectingKMeans(n_clusters=6, random_state=random_seed)
}

def embedding_and_clustering(model_name, cluster_names, use_scaler=False, use_title_only=False):
    print('------------------------------------------------------')
    print('current model: ', model_name)
    
    # Sentence BERT 모델 로드
    model = SentenceTransformer(model_name)

    # 텍스트 feature 추출
    if use_title_only:
        sentence_embeddings = model.encode(df['title'].tolist())
    else:
        sentence_embeddings = model.encode(df['text'].tolist())

    if use_scaler:
        scaler = MinMaxScaler()
        sentence_embeddings = scaler.fit_transform(sentence_embeddings)

    # 추출한 feature를 데이터프레임에 저장
    df_embeddings = pd.DataFrame(sentence_embeddings)

    for cluster_name in cluster_names:
        print('current cluster: ', cluster_name)
        
        cluster_algorithm = cluster_dict[cluster_name]
        
        uses = ''
        if use_scaler or use_title_only:
            uses = f'_scaler{use_scaler}_title{use_title_only}'
        
        current_cname = f'{model_name.split("/")[-1]}_{cluster_name}{uses}'
        df[current_cname] = cluster_algorithm.fit_predict(sentence_embeddings)  
        print('current current_cname: ', current_cname)
        for i in range(6):
            print(f" {i} --------------------------------------")
            temp = df[df[current_cname] == i].head(3).index
            for j in temp:
                print(df['text'][j])
        print("--------------------------------------------")
        

    return df

In [7]:
# df_all_distilroberta_v1 = 
embedding_and_clustering('sentence-transformers/all-distilroberta-v1', ["kmeans_cluster"], True)
# df_all_distilroberta_v1_wt = 
embedding_and_clustering('sentence-transformers/all-distilroberta-v1', ["kmeans_cluster"], True, True)

------------------------------------------------------
current model:  sentence-transformers/all-distilroberta-v1


Loading the tokenizer from the `special_tokens_map.json` and the `added_tokens.json` will be removed in `transformers 5`,  it is kept for forward compatibility, but it is recommended to update your `tokenizer_config.json` by uploading it again. You will see the new `added_tokens_decoder` attribute that will store the relevant information.


current cluster:  kmeans_cluster


/home/ubuntu/anaconda3/envs/use_gpu/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


current current_cname:  all-distilroberta-v1_kmeans_cluster_scalerTrue_titleFalse
 0 --------------------------------------
Only Lovers Left Alive's Tilda Swinton Talks About Almost Quitting Acting and Yasmine Hamdan Performs 'Hal' Live In NYC   (HuffPo Exclusive Videos) authors : Yasmine Hamdan performs 'Hal' which she also sings in the film during a scene when two world-weary vampires begin to heal and find a way to continue living as they remember the power and mystery of creation itself.
Harry #39;s argy-bargy : PRINCE Charles has asked Scotland Yard for an in-depth report on his son Harry #39;s trip to Argentina after reports of excessive drinking and a kidnap plot.
Be on TOP : //www.huffingtonpost.com/entry/be-on-top-amazon-best-sel_b_12508618.html short_description
 1 --------------------------------------
Bruce Lee statue for divided city : In Bosnia, where one man #39;s hero is often another man #39;s villain, some citizens have decided to honour one whom Serbs, Croats and Mus

/home/ubuntu/anaconda3/envs/use_gpu/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


current current_cname:  all-distilroberta-v1_kmeans_cluster_scalerTrue_titleTrue
 0 --------------------------------------
Israel Kills 3 Palestinians in Big Gaza Incursion (Reuters) : Reuters - Israeli forces killed three\Palestinians, including two teenagers, on Wednesday after\storming into the northern Gaza Strip for the third time in as\many months to quell Palestinian rocket fire into Israel.
Bribery Considered, Halliburton Notes Suggest : An internal Halliburton Co. investigation has uncovered handwritten notes suggesting that former employees considered offering bribes to Nigerian officials a decade ago to secure work in a \$5 billion project to build a natural gas liquefaction plant.
Sadr #39;s aide denies entering of Iraqi police into Najaf shrine : BAGHDAD, Aug. 20 (Xinhuanet) -- A top aide to Shiite cleric Moqtada al-Sadr denied a government announcement that Iraqi police entered Imam Ali shrine in Najaf on Friday, Al-Jazeera TV channel reported.
 1 ------------------------

,id,title,contents,text,processed_text,all-distilroberta-v1_kmeans_cluster_scalerTrue_titleFalse,all-distilroberta-v1_kmeans_cluster_scalerTrue_titleTrue
0,NEWS_00000,Spanish coach facing action in race row,MADRID (AFP) - Spanish national team coach Lui...,Spanish coach facing action in race row : MADR...,spanish coach facing action in race row : madr...,3,2
1,NEWS_00001,Bruce Lee statue for divided city,"In Bosnia, where one man #39;s hero is often a...","Bruce Lee statue for divided city : In Bosnia,...","bruce lee statue for divided city : in bosnia,...",1,2
2,NEWS_00002,Only Lovers Left Alive's Tilda Swinton Talks A...,Yasmine Hamdan performs 'Hal' which she also s...,Only Lovers Left Alive's Tilda Swinton Talks A...,only lovers left alive's tilda swinton talks a...,0,4
3,NEWS_00003,Macromedia contributes to eBay Stores,Macromedia has announced a special version of ...,Macromedia contributes to eBay Stores : Macrom...,macromedia contributes to ebay stores : macrom...,5,3
4,NEWS_00004,Qualcomm plans to phone it in on cellular repairs,Over-the-air fixes for cell phones comes to Qu...,Qualcomm plans to phone it in on cellular repa...,qualcomm plans to phone it in on cellular repa...,5,3
...,...,...,...,...,...,...,...
59995,NEWS_59995,"Dolphins Break Through, Rip Rams For First Win",But that #39;s OK. Because after a 31-14 rout ...,"Dolphins Break Through, Rip Rams For First Win...","dolphins break through, rip rams for first win...",3,2
59996,NEWS_59996,"After Steep Drop, Price of Oil Rises",The freefall in oil prices ended Monday on a s...,"After Steep Drop, Price of Oil Rises : The fre...","after steep drop, price of oil rises : the fre...",2,1
59997,NEWS_59997,Pro football: Culpepper puts on a show,To say Daunte Culpepper was a little frustrate...,Pro football: Culpepper puts on a show : To sa...,pro football: culpepper puts on a show : to sa...,3,2
59998,NEWS_59998,Albertsons on the Rebound,The No. 2 grocer reports double-digit gains in...,Albertsons on the Rebound : The No. 2 grocer r...,albertsons on the rebound : the no. grocer re...,2,2


In [8]:
# df_all_MiniLM_L12_v2 = 
embedding_and_clustering('sentence-transformers/all-MiniLM-L12-v2', ["kmeans_cluster", "bkm_cluster", "birch_cluster", "gmm_cluster"])

------------------------------------------------------
current model:  sentence-transformers/all-MiniLM-L12-v2


current cluster:  kmeans_cluster


/home/ubuntu/anaconda3/envs/use_gpu/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


current current_cname:  all-MiniLM-L12-v2_kmeans_cluster
 0 --------------------------------------
Macromedia contributes to eBay Stores : Macromedia has announced a special version of its Contribute website editing application designed to simplify the creation and customisation of eBay Stores.
Qualcomm plans to phone it in on cellular repairs : Over-the-air fixes for cell phones comes to Qualcomm's CDMA.
Thomson to Back Both Blu-ray and HD-DVD : Company, one of the core backers of Blu-ray, will also support its rival format.
 1 --------------------------------------
Obama Marks Anniversary Of 9/11 Attacks With Moment Of Silence authors : We stand as strong as ever.
Republican Congressman Says Trump Should Apologize To Obama And The UK authors : Best not to hold your breath on that one.
Kerry rolls out tax-cut plan for middle class : After two weeks of focusing on Iraq, Democratic presidential challenger John Kerry turned his emphasis to the economy Saturday, delivering what he called 

,id,title,contents,text,processed_text,all-distilroberta-v1_kmeans_cluster_scalerTrue_titleFalse,all-distilroberta-v1_kmeans_cluster_scalerTrue_titleTrue,all-MiniLM-L12-v2_kmeans_cluster,all-MiniLM-L12-v2_bkm_cluster,all-MiniLM-L12-v2_birch_cluster,all-MiniLM-L12-v2_gmm_cluster
0,NEWS_00000,Spanish coach facing action in race row,MADRID (AFP) - Spanish national team coach Lui...,Spanish coach facing action in race row : MADR...,spanish coach facing action in race row : madr...,3,2,4,2,1,5
1,NEWS_00001,Bruce Lee statue for divided city,"In Bosnia, where one man #39;s hero is often a...","Bruce Lee statue for divided city : In Bosnia,...","bruce lee statue for divided city : in bosnia,...",1,2,2,3,3,4
2,NEWS_00002,Only Lovers Left Alive's Tilda Swinton Talks A...,Yasmine Hamdan performs 'Hal' which she also s...,Only Lovers Left Alive's Tilda Swinton Talks A...,only lovers left alive's tilda swinton talks a...,0,4,3,4,5,4
3,NEWS_00003,Macromedia contributes to eBay Stores,Macromedia has announced a special version of ...,Macromedia contributes to eBay Stores : Macrom...,macromedia contributes to ebay stores : macrom...,5,3,0,0,4,0
4,NEWS_00004,Qualcomm plans to phone it in on cellular repairs,Over-the-air fixes for cell phones comes to Qu...,Qualcomm plans to phone it in on cellular repa...,qualcomm plans to phone it in on cellular repa...,5,3,0,0,4,0
...,...,...,...,...,...,...,...,...,...,...,...
59995,NEWS_59995,"Dolphins Break Through, Rip Rams For First Win",But that #39;s OK. Because after a 31-14 rout ...,"Dolphins Break Through, Rip Rams For First Win...","dolphins break through, rip rams for first win...",3,2,4,2,1,5
59996,NEWS_59996,"After Steep Drop, Price of Oil Rises",The freefall in oil prices ended Monday on a s...,"After Steep Drop, Price of Oil Rises : The fre...","after steep drop, price of oil rises : the fre...",2,1,5,1,0,1
59997,NEWS_59997,Pro football: Culpepper puts on a show,To say Daunte Culpepper was a little frustrate...,Pro football: Culpepper puts on a show : To sa...,pro football: culpepper puts on a show : to sa...,3,2,4,2,1,5
59998,NEWS_59998,Albertsons on the Rebound,The No. 2 grocer reports double-digit gains in...,Albertsons on the Rebound : The No. 2 grocer r...,albertsons on the rebound : the no. grocer re...,2,2,5,1,0,1


In [9]:
# df_all_mpnet_base_v2 = 
embedding_and_clustering('sentence-transformers/all-mpnet-base-v2', ["kmeans_cluster"])

------------------------------------------------------
current model:  sentence-transformers/all-mpnet-base-v2


current cluster:  kmeans_cluster


/home/ubuntu/anaconda3/envs/use_gpu/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


current current_cname:  all-mpnet-base-v2_kmeans_cluster
 0 --------------------------------------
Macromedia contributes to eBay Stores : Macromedia has announced a special version of its Contribute website editing application designed to simplify the creation and customisation of eBay Stores.
Qualcomm plans to phone it in on cellular repairs : Over-the-air fixes for cell phones comes to Qualcomm's CDMA.
Thomson to Back Both Blu-ray and HD-DVD : Company, one of the core backers of Blu-ray, will also support its rival format.
 1 --------------------------------------
Spanish coach facing action in race row : MADRID (AFP) - Spanish national team coach Luis Aragones faces a formal investigation after Spain #39;s Football Federation decided to open disciplinary proceedings over racist comments about Thierry Henry of France and Arsenal.
Time to Talk Baseball : It's time to talk about the serious risks and potential benefits of building an expensive ballpark in Washington.
GAME DAY PREVIEW 

,id,title,contents,text,processed_text,all-distilroberta-v1_kmeans_cluster_scalerTrue_titleFalse,all-distilroberta-v1_kmeans_cluster_scalerTrue_titleTrue,all-MiniLM-L12-v2_kmeans_cluster,all-MiniLM-L12-v2_bkm_cluster,all-MiniLM-L12-v2_birch_cluster,all-MiniLM-L12-v2_gmm_cluster,all-mpnet-base-v2_kmeans_cluster
0,NEWS_00000,Spanish coach facing action in race row,MADRID (AFP) - Spanish national team coach Lui...,Spanish coach facing action in race row : MADR...,spanish coach facing action in race row : madr...,3,2,4,2,1,5,1
1,NEWS_00001,Bruce Lee statue for divided city,"In Bosnia, where one man #39;s hero is often a...","Bruce Lee statue for divided city : In Bosnia,...","bruce lee statue for divided city : in bosnia,...",1,2,2,3,3,4,2
2,NEWS_00002,Only Lovers Left Alive's Tilda Swinton Talks A...,Yasmine Hamdan performs 'Hal' which she also s...,Only Lovers Left Alive's Tilda Swinton Talks A...,only lovers left alive's tilda swinton talks a...,0,4,3,4,5,4,5
3,NEWS_00003,Macromedia contributes to eBay Stores,Macromedia has announced a special version of ...,Macromedia contributes to eBay Stores : Macrom...,macromedia contributes to ebay stores : macrom...,5,3,0,0,4,0,0
4,NEWS_00004,Qualcomm plans to phone it in on cellular repairs,Over-the-air fixes for cell phones comes to Qu...,Qualcomm plans to phone it in on cellular repa...,qualcomm plans to phone it in on cellular repa...,5,3,0,0,4,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
59995,NEWS_59995,"Dolphins Break Through, Rip Rams For First Win",But that #39;s OK. Because after a 31-14 rout ...,"Dolphins Break Through, Rip Rams For First Win...","dolphins break through, rip rams for first win...",3,2,4,2,1,5,1
59996,NEWS_59996,"After Steep Drop, Price of Oil Rises",The freefall in oil prices ended Monday on a s...,"After Steep Drop, Price of Oil Rises : The fre...","after steep drop, price of oil rises : the fre...",2,1,5,1,0,1,4
59997,NEWS_59997,Pro football: Culpepper puts on a show,To say Daunte Culpepper was a little frustrate...,Pro football: Culpepper puts on a show : To sa...,pro football: culpepper puts on a show : to sa...,3,2,4,2,1,5,1
59998,NEWS_59998,Albertsons on the Rebound,The No. 2 grocer reports double-digit gains in...,Albertsons on the Rebound : The No. 2 grocer r...,albertsons on the rebound : the no. grocer re...,2,2,5,1,0,1,4


In [10]:
# df_all_MiniLM_L6_v2 = 
embedding_and_clustering('sentence-transformers/all-MiniLM-L6-v2', ["agglo_cluster"])

------------------------------------------------------
current model:  sentence-transformers/all-MiniLM-L6-v2
current cluster:  agglo_cluster
current current_cname:  all-MiniLM-L6-v2_agglo_cluster
 0 --------------------------------------
Time to Talk Baseball : It's time to talk about the serious risks and potential benefits of building an expensive ballpark in Washington.
Obama Administration Helps Wall Street Criminals Dodge Accountability authors : Obama administration proposal would aid big banks that have pleaded guilty to felony antitrust charges.
Congress Spikes Handout For Private Equity authors : A few Wall Street firms almost won big.
 1 --------------------------------------
Spanish coach facing action in race row : MADRID (AFP) - Spanish national team coach Luis Aragones faces a formal investigation after Spain #39;s Football Federation decided to open disciplinary proceedings over racist comments about Thierry Henry of France and Arsenal.
GAME DAY PREVIEW Game time: 6:00 

,id,title,contents,text,processed_text,all-distilroberta-v1_kmeans_cluster_scalerTrue_titleFalse,all-distilroberta-v1_kmeans_cluster_scalerTrue_titleTrue,all-MiniLM-L12-v2_kmeans_cluster,all-MiniLM-L12-v2_bkm_cluster,all-MiniLM-L12-v2_birch_cluster,all-MiniLM-L12-v2_gmm_cluster,all-mpnet-base-v2_kmeans_cluster,all-MiniLM-L6-v2_agglo_cluster
0,NEWS_00000,Spanish coach facing action in race row,MADRID (AFP) - Spanish national team coach Lui...,Spanish coach facing action in race row : MADR...,spanish coach facing action in race row : madr...,3,2,4,2,1,5,1,1
1,NEWS_00001,Bruce Lee statue for divided city,"In Bosnia, where one man #39;s hero is often a...","Bruce Lee statue for divided city : In Bosnia,...","bruce lee statue for divided city : in bosnia,...",1,2,2,3,3,4,2,5
2,NEWS_00002,Only Lovers Left Alive's Tilda Swinton Talks A...,Yasmine Hamdan performs 'Hal' which she also s...,Only Lovers Left Alive's Tilda Swinton Talks A...,only lovers left alive's tilda swinton talks a...,0,4,3,4,5,4,5,5
3,NEWS_00003,Macromedia contributes to eBay Stores,Macromedia has announced a special version of ...,Macromedia contributes to eBay Stores : Macrom...,macromedia contributes to ebay stores : macrom...,5,3,0,0,4,0,0,4
4,NEWS_00004,Qualcomm plans to phone it in on cellular repairs,Over-the-air fixes for cell phones comes to Qu...,Qualcomm plans to phone it in on cellular repa...,qualcomm plans to phone it in on cellular repa...,5,3,0,0,4,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,NEWS_59995,"Dolphins Break Through, Rip Rams For First Win",But that #39;s OK. Because after a 31-14 rout ...,"Dolphins Break Through, Rip Rams For First Win...","dolphins break through, rip rams for first win...",3,2,4,2,1,5,1,1
59996,NEWS_59996,"After Steep Drop, Price of Oil Rises",The freefall in oil prices ended Monday on a s...,"After Steep Drop, Price of Oil Rises : The fre...","after steep drop, price of oil rises : the fre...",2,1,5,1,0,1,4,0
59997,NEWS_59997,Pro football: Culpepper puts on a show,To say Daunte Culpepper was a little frustrate...,Pro football: Culpepper puts on a show : To sa...,pro football: culpepper puts on a show : to sa...,3,2,4,2,1,5,1,1
59998,NEWS_59998,Albertsons on the Rebound,The No. 2 grocer reports double-digit gains in...,Albertsons on the Rebound : The No. 2 grocer r...,albertsons on the rebound : the no. grocer re...,2,2,5,1,0,1,4,0


In [11]:
# df_paraphrase_MiniLM_L6_v2 = 
embedding_and_clustering('sentence-transformers/paraphrase-MiniLM-L6-v2', ["birch_cluster", "gmm_cluster"])

------------------------------------------------------
current model:  sentence-transformers/paraphrase-MiniLM-L6-v2


current cluster:  birch_cluster
current current_cname:  paraphrase-MiniLM-L6-v2_birch_cluster
 0 --------------------------------------
Macromedia contributes to eBay Stores : Macromedia has announced a special version of its Contribute website editing application designed to simplify the creation and customisation of eBay Stores.
Qualcomm plans to phone it in on cellular repairs : Over-the-air fixes for cell phones comes to Qualcomm's CDMA.
Thomson to Back Both Blu-ray and HD-DVD : Company, one of the core backers of Blu-ray, will also support its rival format.
 1 --------------------------------------
Israel Kills 3 Palestinians in Big Gaza Incursion (Reuters) : Reuters - Israeli forces killed three\Palestinians, including two teenagers, on Wednesday after\storming into the northern Gaza Strip for the third time in as\many months to quell Palestinian rocket fire into Israel.
Memo To EPA Chief Pruitt : //www.huffingtonpost.com/entry/memo-to-epa-chief-pruitt-lets-end-subsidies-for-foss

,id,title,contents,text,processed_text,all-distilroberta-v1_kmeans_cluster_scalerTrue_titleFalse,all-distilroberta-v1_kmeans_cluster_scalerTrue_titleTrue,all-MiniLM-L12-v2_kmeans_cluster,all-MiniLM-L12-v2_bkm_cluster,all-MiniLM-L12-v2_birch_cluster,all-MiniLM-L12-v2_gmm_cluster,all-mpnet-base-v2_kmeans_cluster,all-MiniLM-L6-v2_agglo_cluster,paraphrase-MiniLM-L6-v2_birch_cluster,paraphrase-MiniLM-L6-v2_gmm_cluster
0,NEWS_00000,Spanish coach facing action in race row,MADRID (AFP) - Spanish national team coach Lui...,Spanish coach facing action in race row : MADR...,spanish coach facing action in race row : madr...,3,2,4,2,1,5,1,1,2,3
1,NEWS_00001,Bruce Lee statue for divided city,"In Bosnia, where one man #39;s hero is often a...","Bruce Lee statue for divided city : In Bosnia,...","bruce lee statue for divided city : in bosnia,...",1,2,2,3,3,4,2,5,3,5
2,NEWS_00002,Only Lovers Left Alive's Tilda Swinton Talks A...,Yasmine Hamdan performs 'Hal' which she also s...,Only Lovers Left Alive's Tilda Swinton Talks A...,only lovers left alive's tilda swinton talks a...,0,4,3,4,5,4,5,5,3,4
3,NEWS_00003,Macromedia contributes to eBay Stores,Macromedia has announced a special version of ...,Macromedia contributes to eBay Stores : Macrom...,macromedia contributes to ebay stores : macrom...,5,3,0,0,4,0,0,4,0,0
4,NEWS_00004,Qualcomm plans to phone it in on cellular repairs,Over-the-air fixes for cell phones comes to Qu...,Qualcomm plans to phone it in on cellular repa...,qualcomm plans to phone it in on cellular repa...,5,3,0,0,4,0,0,4,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,NEWS_59995,"Dolphins Break Through, Rip Rams For First Win",But that #39;s OK. Because after a 31-14 rout ...,"Dolphins Break Through, Rip Rams For First Win...","dolphins break through, rip rams for first win...",3,2,4,2,1,5,1,1,2,3
59996,NEWS_59996,"After Steep Drop, Price of Oil Rises",The freefall in oil prices ended Monday on a s...,"After Steep Drop, Price of Oil Rises : The fre...","after steep drop, price of oil rises : the fre...",2,1,5,1,0,1,4,0,0,1
59997,NEWS_59997,Pro football: Culpepper puts on a show,To say Daunte Culpepper was a little frustrate...,Pro football: Culpepper puts on a show : To sa...,pro football: culpepper puts on a show : to sa...,3,2,4,2,1,5,1,1,2,3
59998,NEWS_59998,Albertsons on the Rebound,The No. 2 grocer reports double-digit gains in...,Albertsons on the Rebound : The No. 2 grocer r...,albertsons on the rebound : the no. grocer re...,2,2,5,1,0,1,4,0,0,1


In [12]:
# df_paraphrase_MiniLM_L12_v2= 
embedding_and_clustering('sentence-transformers/paraphrase-MiniLM-L12-v2', ["kmeans_cluster"])

------------------------------------------------------
current model:  sentence-transformers/paraphrase-MiniLM-L12-v2


current cluster:  kmeans_cluster


/home/ubuntu/anaconda3/envs/use_gpu/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


current current_cname:  paraphrase-MiniLM-L12-v2_kmeans_cluster
 0 --------------------------------------
Only Lovers Left Alive's Tilda Swinton Talks About Almost Quitting Acting and Yasmine Hamdan Performs 'Hal' Live In NYC   (HuffPo Exclusive Videos) authors : Yasmine Hamdan performs 'Hal' which she also sings in the film during a scene when two world-weary vampires begin to heal and find a way to continue living as they remember the power and mystery of creation itself.
Read Live Updates From The South Carolina Democratic Primary authors : South Carolina Democrats are making their pick on Saturday.
It's Not As Easy As You Think To Spot A Gerrymandered Map authors : Our electoral districts have really weird shapes. Figuring out if they're gerrymandered takes some deeper digging.
 1 --------------------------------------
Spanish coach facing action in race row : MADRID (AFP) - Spanish national team coach Luis Aragones faces a formal investigation after Spain #39;s Football Federation

,id,title,contents,text,processed_text,all-distilroberta-v1_kmeans_cluster_scalerTrue_titleFalse,all-distilroberta-v1_kmeans_cluster_scalerTrue_titleTrue,all-MiniLM-L12-v2_kmeans_cluster,all-MiniLM-L12-v2_bkm_cluster,all-MiniLM-L12-v2_birch_cluster,all-MiniLM-L12-v2_gmm_cluster,all-mpnet-base-v2_kmeans_cluster,all-MiniLM-L6-v2_agglo_cluster,paraphrase-MiniLM-L6-v2_birch_cluster,paraphrase-MiniLM-L6-v2_gmm_cluster,paraphrase-MiniLM-L12-v2_kmeans_cluster
0,NEWS_00000,Spanish coach facing action in race row,MADRID (AFP) - Spanish national team coach Lui...,Spanish coach facing action in race row : MADR...,spanish coach facing action in race row : madr...,3,2,4,2,1,5,1,1,2,3,1
1,NEWS_00001,Bruce Lee statue for divided city,"In Bosnia, where one man #39;s hero is often a...","Bruce Lee statue for divided city : In Bosnia,...","bruce lee statue for divided city : in bosnia,...",1,2,2,3,3,4,2,5,3,5,2
2,NEWS_00002,Only Lovers Left Alive's Tilda Swinton Talks A...,Yasmine Hamdan performs 'Hal' which she also s...,Only Lovers Left Alive's Tilda Swinton Talks A...,only lovers left alive's tilda swinton talks a...,0,4,3,4,5,4,5,5,3,4,0
3,NEWS_00003,Macromedia contributes to eBay Stores,Macromedia has announced a special version of ...,Macromedia contributes to eBay Stores : Macrom...,macromedia contributes to ebay stores : macrom...,5,3,0,0,4,0,0,4,0,0,3
4,NEWS_00004,Qualcomm plans to phone it in on cellular repairs,Over-the-air fixes for cell phones comes to Qu...,Qualcomm plans to phone it in on cellular repa...,qualcomm plans to phone it in on cellular repa...,5,3,0,0,4,0,0,4,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,NEWS_59995,"Dolphins Break Through, Rip Rams For First Win",But that #39;s OK. Because after a 31-14 rout ...,"Dolphins Break Through, Rip Rams For First Win...","dolphins break through, rip rams for first win...",3,2,4,2,1,5,1,1,2,3,1
59996,NEWS_59996,"After Steep Drop, Price of Oil Rises",The freefall in oil prices ended Monday on a s...,"After Steep Drop, Price of Oil Rises : The fre...","after steep drop, price of oil rises : the fre...",2,1,5,1,0,1,4,0,0,1,2
59997,NEWS_59997,Pro football: Culpepper puts on a show,To say Daunte Culpepper was a little frustrate...,Pro football: Culpepper puts on a show : To sa...,pro football: culpepper puts on a show : to sa...,3,2,4,2,1,5,1,1,2,3,1
59998,NEWS_59998,Albertsons on the Rebound,The No. 2 grocer reports double-digit gains in...,Albertsons on the Rebound : The No. 2 grocer r...,albertsons on the rebound : the no. grocer re...,2,2,5,1,0,1,4,0,0,1,3


결과를 보고 각자 매핑해줍니다.

In [13]:
# 1

print(df.columns)

mapping_dict = {}

for col in df.columns:
    mapping_dict[col] = {
                            0: 0,
                            1: 1,
                            2: 2,
                            3: 3,
                            4: 4,
                            5: 5
                        }
    
print(mapping_dict)

Index(['id', 'title', 'contents', 'text', 'processed_text',
       'all-distilroberta-v1_kmeans_cluster_scalerTrue_titleFalse',
       'all-distilroberta-v1_kmeans_cluster_scalerTrue_titleTrue',
       'all-MiniLM-L12-v2_kmeans_cluster', 'all-MiniLM-L12-v2_bkm_cluster',
       'all-MiniLM-L12-v2_birch_cluster', 'all-MiniLM-L12-v2_gmm_cluster',
       'all-mpnet-base-v2_kmeans_cluster', 'all-MiniLM-L6-v2_agglo_cluster',
       'paraphrase-MiniLM-L6-v2_birch_cluster',
       'paraphrase-MiniLM-L6-v2_gmm_cluster',
       'paraphrase-MiniLM-L12-v2_kmeans_cluster'],
      dtype='object')
{'id': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5}, 'title': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5}, 'contents': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5}, 'text': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5}, 'processed_text': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5}, 'all-distilroberta-v1_kmeans_cluster_scalerTrue_titleFalse': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5}, 'all-distilroberta-v1_kmeans_cluster_scalerTrue_titleTrue': {0: 0, 

csv 를 앙상블합니다.

In [14]:
# 2
mapping_dict = {
    'all-distilroberta-v1_kmeans_cluster_scalerTrue_titleFalse': {0: 1, 1: 5, 2: 0, 3: 3, 4: 2, 5: 4}
    , 'all-distilroberta-v1_kmeans_cluster_scalerTrue_titleTrue': {0: 5, 1: 0, 2: 3, 3: 4, 4: 1, 5: 2} 
    , 'all-MiniLM-L12-v2_kmeans_cluster': {0: 4, 1: 2, 2: 5, 3: 1, 4: 3, 5: 0}
    , 'all-MiniLM-L12-v2_bkm_cluster': {0: 4, 1: 0, 2: 3, 3: 5, 4: 1, 5: 2}
    , 'all-MiniLM-L12-v2_birch_cluster': {0: 0, 1: 3, 2: 2, 3: 5, 4: 4, 5: 1}
    , 'all-MiniLM-L12-v2_gmm_cluster': {0: 4, 1: 0, 2: 5, 3: 2, 4: 1, 5: 3}
    , 'all-mpnet-base-v2_kmeans_cluster': {0: 4, 1: 3, 2: 5, 3: 2, 4: 0, 5: 1}
    , 'all-MiniLM-L6-v2_agglo_cluster': {0: 0, 1: 3, 2: 2, 3: 5, 4: 4, 5: 1}
    , 'paraphrase-MiniLM-L6-v2_birch_cluster': {0: 4, 1: 5, 2: 3, 3: 1, 4: 0, 5: 2}
    , 'paraphrase-MiniLM-L6-v2_gmm_cluster': {0: 4, 1: 0, 2: 5, 3: 3, 4: 2, 5: 1}
    , 'paraphrase-MiniLM-L12-v2_kmeans_cluster': {0: 5, 1: 3, 2: 1, 3: 4, 4: 0, 5: 2}
}

In [15]:
# column name : csv name
mapping_csv_dict = {
    "all-distilroberta-v1_kmeans_cluster_scalerTrue_titleTrue": "st_submit_knn_allbert_scaler_title.csv", # sentence-transformers/all-distilroberta-v1
    "all-distilroberta-v1_kmeans_cluster_scalerTrue_titleFalse" :"st_submit_knn_allbert_scaler.csv", 
    "all-MiniLM-L12-v2_kmeans_cluster":"st_submit_knn_mini12.csv", # sentence-transformers/all-MiniLM-L12-v2
    "all-MiniLM-L12-v2_bkm_cluster":'st_submit_bkm_mini12.csv', 
    "all-MiniLM-L12-v2_birch_cluster":"st_submit_birch_mini12.csv", 
    'all-mpnet-base-v2_kmeans_cluster':"st_submit_knn_mpnet.csv", # sentence-transformers/all-mpnet-base-v2
    'all-MiniLM-L6-v2_agglo_cluster':'st_submit_agglomerative.csv', # sentence-transformers/all-MiniLM-L6-v2
    'paraphrase-MiniLM-L6-v2_birch_cluster':'st_submit_birch_pminilm.csv', # sentence-transformers/paraphrase-MiniLM-L6-v2
    'paraphrase-MiniLM-L6-v2_gmm_cluster':'st_submit_gmm_pminilm.csv', 
    "paraphrase-MiniLM-L12-v2_kmeans_cluster":'st_submit_knn_pmini12.csv', # sentence-transformers/paraphrase-MiniLM-L12-v2
    "all-MiniLM-L12-v2_gmm_cluster":'st_submit_gmm_mini12.csv' # sentence-transformers/all-MiniLM-L12-v2

}

In [16]:
FILE_PATH, SAVE_PATH

('/home/ubuntu/use_gpu/classification',
 '/home/ubuntu/use_gpu/classification/submit')

In [17]:
sample = pd.read_csv(os.path.join(FILE_PATH ,'sample_submission.csv'))

for column_name, csv_name in mapping_csv_dict.items():

    current_mapping_dict = mapping_dict[column_name]
    df['mapping'] = df[column_name].apply(lambda x: current_mapping_dict[x])
    sample['category'] = df['mapping'].values
    sample.to_csv(os.path.join(SAVE_PATH, csv_name), index=False)
    
    print("--------------------------------------------")
    print(f"saved {os.path.join(SAVE_PATH, csv_name)}")

--------------------------------------------
saved /home/ubuntu/use_gpu/classification/submit/st_submit_knn_allbert_scaler_title.csv
--------------------------------------------
saved /home/ubuntu/use_gpu/classification/submit/st_submit_knn_allbert_scaler.csv
--------------------------------------------
saved /home/ubuntu/use_gpu/classification/submit/st_submit_knn_mini12.csv
--------------------------------------------
saved /home/ubuntu/use_gpu/classification/submit/st_submit_bkm_mini12.csv
--------------------------------------------
saved /home/ubuntu/use_gpu/classification/submit/st_submit_birch_mini12.csv
--------------------------------------------
saved /home/ubuntu/use_gpu/classification/submit/st_submit_knn_mpnet.csv
--------------------------------------------
saved /home/ubuntu/use_gpu/classification/submit/st_submit_agglomerative.csv
--------------------------------------------
saved /home/ubuntu/use_gpu/classification/submit/st_submit_birch_pminilm.csv
-------------------

In [18]:
# 2

csv_dict = {
    'ensemble_result_wo3_w24ensemble_gmm.csv': ["st_submit_knn_allbert_scaler.csv", "st_submit_knn_mini12.csv", "st_submit_knn_mpnet.csv", "st_submit_birch_mini12.csv", "st_submit_knn_allbert_scaler_title.csv", 'st_submit_bkm_mini12.csv' , 'st_submit_agglomerative.csv', 'st_submit_gmm_mini12.csv']
    ,'ensemble_result_wo3_w24ensemble_wo5.csv': ['st_submit_knn_allbert_scaler.csv', 'st_submit_knn_mini12.csv', 'st_submit_knn_mpnet.csv', 'st_submit_birch_mini12.csv', 'st_submit_knn_allbert_scaler_title.csv', 'st_submit_agglomerative.csv']
    ,'ensemble_result_wo3_w24ensemble_wo2.csv': ['st_submit_knn_allbert_scaler.csv', 'st_submit_knn_mini12.csv', 'st_submit_birch_mini12.csv', 'st_submit_knn_allbert_scaler_title.csv', 'st_submit_bkm_mini12.csv', 'st_submit_agglomerative.csv']
    ,'ensemble_result_wo3_w24ensemble_wo1.csv': ['st_submit_knn_allbert_scaler.csv', 'st_submit_knn_mpnet.csv', 'st_submit_birch_mini12.csv', 'st_submit_knn_allbert_scaler_title.csv', 'st_submit_bkm_mini12.csv', 'st_submit_agglomerative.csv']
    ,'ensemble_result_wo3_w7_enall2.csv': ["st_submit_knn_allbert_scaler.csv", "st_submit_knn_mini12.csv", "st_submit_knn_mpnet.csv", "st_submit_birch_mini12.csv", "st_submit_knn_allbert_scaler_title.csv", 'st_submit_bkm_mini12.csv' , 'st_submit_agglomerative.csv', 'ensemble_result_wo3_w24ensemble_wo2.csv', 'ensemble_result_wo3_w24ensemble_wo1.csv', 'ensemble_result_wo3_w24ensemble_gmm.csv', 'ensemble_result_wo3_w24ensemble_wo5.csv']
    ,'ensemble_result_wo3_w7_enall4.csv': ["st_submit_knn_allbert_scaler.csv", "st_submit_knn_mini12.csv", "st_submit_knn_mpnet.csv", "st_submit_birch_mini12.csv", "st_submit_knn_allbert_scaler_title.csv", 'st_submit_bkm_mini12.csv' , 'st_submit_agglomerative.csv', 'ensemble_result_wo3_w24ensemble_wo2.csv', 'ensemble_result_wo3_w24ensemble_wo1.csv', 'ensemble_result_wo3_w24ensemble_gmm.csv'] 
    ,'ensemble_result_wo3_w7_en0.csv': ['st_submit_knn_allbert_scaler.csv', 'st_submit_knn_mini12.csv', 'st_submit_knn_mpnet.csv', 'st_submit_birch_mini12.csv', 'st_submit_knn_allbert_scaler_title.csv', 'st_submit_bkm_mini12.csv', 'st_submit_agglomerative.csv', 'ensemble_result_wo3_w24ensemble_wo2.csv', 'ensemble_result_wo3_w24ensemble_wo1.csv']
    }

# 결과를 CSV 파일로 저장
sample = pd.read_csv(os.path.join(FILE_PATH ,'sample_submission.csv'))

for key, file_paths in csv_dict.items():
    print("--------------------------------------------")
    print(key)

    # 각 파일에서 라벨 추출
    all_labels = []
    for file_path in file_paths:
        df = pd.read_csv(os.path.join(SAVE_PATH, file_path))
        labels = df['category'].tolist()  
        all_labels.append(labels)

    # 각 라벨 리스트에서 가장 빈번한 라벨을 선택 (다수결 투표)
    final_labels = []
    for i in range(len(all_labels[0])):
        label_votes = [labels[i] for labels in all_labels]
        majority_label = Counter(label_votes).most_common(1)[0][0]
        final_labels.append(majority_label)

    sample['category'] = final_labels
    sample.to_csv(os.path.join(SAVE_PATH, key), index=False)

--------------------------------------------
ensemble_result_wo3_w24ensemble_gmm.csv
--------------------------------------------
ensemble_result_wo3_w24ensemble_wo5.csv
--------------------------------------------
ensemble_result_wo3_w24ensemble_wo2.csv
--------------------------------------------
ensemble_result_wo3_w24ensemble_wo1.csv
--------------------------------------------
ensemble_result_wo3_w7_enall2.csv
--------------------------------------------
ensemble_result_wo3_w7_enall4.csv
--------------------------------------------
ensemble_result_wo3_w7_en0.csv


만들어진 csv 들을 다같이 다시 앙상블합니다.

In [19]:
# 3


# CSV 파일 목록
file_paths = [
    "st_submit_knn_allbert_scaler_title.csv", # sentence-transformers/all-distilroberta-v1
    "st_submit_knn_allbert_scaler.csv", 
    "st_submit_knn_mini12.csv", # sentence-transformers/all-MiniLM-L12-v2
    'st_submit_bkm_mini12.csv', 
    "st_submit_birch_mini12.csv", 
    "st_submit_knn_mpnet.csv", # sentence-transformers/all-mpnet-base-v2
    'st_submit_agglomerative.csv', # sentence-transformers/all-MiniLM-L6-v2
    'st_submit_birch_pminilm.csv', # sentence-transformers/paraphrase-MiniLM-L6-v2
    'st_submit_gmm_pminilm.csv', 
    'st_submit_knn_pmini12.csv', # sentence-transformers/paraphrase-MiniLM-L12-v2

    'ensemble_result_wo3_w24ensemble_wo2.csv',  # ['st_submit_knn_allbert_scaler.csv', 'st_submit_knn_mini12.csv', 'st_submit_birch_mini12.csv', 'st_submit_knn_allbert_scaler_title.csv', 'st_submit_bkm_mini12.csv', 'st_submit_agglomerative.csv']
    'ensemble_result_wo3_w24ensemble_wo1.csv',  # ['st_submit_knn_allbert_scaler.csv', 'st_submit_knn_mpnet.csv', 'st_submit_birch_mini12.csv', 'st_submit_knn_allbert_scaler_title.csv', 'st_submit_bkm_mini12.csv', 'st_submit_agglomerative.csv']
    'ensemble_result_wo3_w7_enall2.csv',        # ["st_submit_knn_allbert_scaler.csv", "st_submit_knn_mini12.csv", "st_submit_knn_mpnet.csv", "st_submit_birch_mini12.csv", "st_submit_knn_allbert_scaler_title.csv", 'st_submit_bkm_mini12.csv' , 'st_submit_agglomerative.csv', 'ensemble_result_wo3_w24ensemble_wo2.csv', 'ensemble_result_wo3_w24ensemble_wo1.csv', 'ensemble_result_wo3_w24ensemble_gmm.csv', 'ensemble_result_wo3_w24ensemble_wo5.csv']
    'ensemble_result_wo3_w7_enall4.csv',         # ["st_submit_knn_allbert_scaler.csv", "st_submit_knn_mini12.csv", "st_submit_knn_mpnet.csv", "st_submit_birch_mini12.csv", "st_submit_knn_allbert_scaler_title.csv", 'st_submit_bkm_mini12.csv' , 'st_submit_agglomerative.csv', ]'ensemble_result_wo3_w24ensemble_wo2.csv', 'ensemble_result_wo3_w24ensemble_wo1.csv', 'ensemble_result_wo3_w24ensemble_gmm.csv'] 
    'ensemble_result_wo3_w7_en0.csv'            #  ['st_submit_knn_allbert_scaler.csv', 'st_submit_knn_mini12.csv', 'st_submit_knn_mpnet.csv', 'st_submit_birch_mini12.csv', 'st_submit_knn_allbert_scaler_title.csv', 'st_submit_bkm_mini12.csv', 'st_submit_agglomerative.csv', 'ensemble_result_wo3_w24ensemble_wo2.csv', 'ensemble_result_wo3_w24ensemble_wo1.csv']
]

print('file_paths: ', file_paths)

# 각 파일에서 라벨 추출
all_labels = []
for file_path in file_paths:
    df = pd.read_csv(os.path.join(SAVE_PATH, file_path))
    labels = df['category'].tolist()  
    all_labels.append(labels)

# 각 라벨 리스트에서 가장 빈번한 라벨을 선택 (다수결 투표)
final_labels = []
for i in range(len(all_labels[0])):
    label_votes = [labels[i] for labels in all_labels]
    majority_label = Counter(label_votes).most_common(1)[0][0]
    final_labels.append(majority_label)

# 결과를 CSV 파일로 저장
sample = pd.read_csv(os.path.join(FILE_PATH ,'sample_submission.csv'))
sample['category'] = final_labels
sample.to_csv(os.path.join(SAVE_PATH, f'ensemble_submit.csv'), index=False)

print('saved ensemble_submit.csv')

file_paths:  ['st_submit_knn_allbert_scaler_title.csv', 'st_submit_knn_allbert_scaler.csv', 'st_submit_knn_mini12.csv', 'st_submit_bkm_mini12.csv', 'st_submit_birch_mini12.csv', 'st_submit_knn_mpnet.csv', 'st_submit_agglomerative.csv', 'st_submit_birch_pminilm.csv', 'st_submit_gmm_pminilm.csv', 'st_submit_knn_pmini12.csv', 'ensemble_result_wo3_w24ensemble_wo2.csv', 'ensemble_result_wo3_w24ensemble_wo1.csv', 'ensemble_result_wo3_w7_enall2.csv', 'ensemble_result_wo3_w7_enall4.csv', 'ensemble_result_wo3_w7_en0.csv']
saved ensemble_submit.csv
